In [2]:
import pandas as pd
from transformers import pipeline
from sklearn.metrics import classification_report

/home/abitporu/Documents/azmina/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-04 12:03:07.820197: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-04 12:03:07.821075: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-04 12:03:07.857095: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-04 12:03:08.001650: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instruct

In [3]:
DATA_PATH = "../../data/"

INPUT_TRAIN = DATA_PATH + "train.csv"

INPUT_TEST = DATA_PATH + "test.csv"

In [4]:
train = pd.read_csv(INPUT_TRAIN)
test = pd.read_csv(INPUT_TEST)

In [5]:
set(train.macro_tema.unique()) == set(test.macro_tema.unique())

True

In [6]:
labels = test.macro_tema.unique()
labels

array(['direitos sexuais e reprodutivos', 'dignidade sexual',
       'violencia contra a mulher', 'economia', 'maternidade',
       'direitos sociais', 'lei maria da penha', 'genero', 'politica',
       'feminicidio', 'outros'], dtype=object)

In [7]:
pipe = pipeline(model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")

def classify_summary(summary,labels=labels):
    result = pipe(summary, candidate_labels=labels)
    return result["labels"][0]


In [8]:
string_test = "Institui a Campanha Nacional de Combate ao Assédio e à Violência Sexual \
                praticados contra crianças e adolescentes nos locais de prática desportiva e dá outras providências"

classify_summary(string_test)

'violencia contra a mulher'

In [8]:
test["mdeberta_pred"] = test.resumo.apply(classify_summary)

test["mdeberta_pred"].value_counts()

mdeberta_pred
violencia contra a mulher          94
genero                             41
lei maria da penha                 40
maternidade                        32
direitos sexuais e reprodutivos    32
politica                           26
direitos sociais                   23
outros                             22
dignidade sexual                    8
feminicidio                         3
Name: count, dtype: int64

In [11]:
print(classification_report(test['macro_tema'], test['mdeberta_pred']))

                                 precision    recall  f1-score   support

               dignidade sexual       0.62      0.17      0.27        29
direitos sexuais e reprodutivos       0.25      0.26      0.25        31
               direitos sociais       0.13      0.10      0.11        30
                       economia       0.00      0.00      0.00        24
                    feminicidio       0.67      0.25      0.36         8
                         genero       0.15      0.32      0.20        19
             lei maria da penha       0.57      0.66      0.61        35
                    maternidade       0.56      0.56      0.56        32
                         outros       0.00      0.00      0.00        10
                       politica       0.08      0.15      0.10        13
      violencia contra a mulher       0.53      0.56      0.54        90

                       accuracy                           0.36       321
                      macro avg       0.32      0

/home/abitporu/Documents/azmina/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/abitporu/Documents/azmina/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/abitporu/Documents/azmina/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.

In [9]:
pipe = pipeline(model="facebook/bart-large-mnli")

def classify_summary(summary,labels=labels):
    result = pipe(summary, candidate_labels=labels)
    return result["labels"][0]

test["bart_pred"] = test.resumo.apply(classify_summary)

test["bart_pred"].value_counts()

bart_pred
maternidade                        84
violencia contra a mulher          67
direitos sexuais e reprodutivos    38
lei maria da penha                 34
dignidade sexual                   28
direitos sociais                   25
genero                             23
outros                             16
politica                            4
feminicidio                         2
Name: count, dtype: int64

In [11]:
print(classification_report(test['macro_tema'], test['bart_pred']))


                                 precision    recall  f1-score   support

               dignidade sexual       0.29      0.28      0.28        29
direitos sexuais e reprodutivos       0.21      0.26      0.23        31
               direitos sociais       0.08      0.07      0.07        30
                       economia       0.00      0.00      0.00        24
                    feminicidio       1.00      0.25      0.40         8
                         genero       0.04      0.05      0.05        19
             lei maria da penha       0.59      0.57      0.58        35
                    maternidade       0.24      0.62      0.34        32
                         outros       0.00      0.00      0.00        10
                       politica       0.75      0.23      0.35        13
      violencia contra a mulher       0.54      0.40      0.46        90

                       accuracy                           0.31       321
                      macro avg       0.34      0

/home/abitporu/Documents/azmina/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/abitporu/Documents/azmina/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/abitporu/Documents/azmina/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.